In [1]:
from astropy import units as u                                                                                                             
from astropy.nddata import CCDData                                                                                                         
from astropy.io import fits
import ccdproc
from ccdproc import Combiner        
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import glob
%matplotlib inline
plt.style.use("dark_background")
dr = "/home/fischwagen/prg/astronomy/BDF/"
plt.rcParams['figure.figsize'] = [14, 7]

In [4]:
def trim_fits(arr):
    shape = arr.shape
    ccd = CCDData(arr, unit=u.adu)
    ccd = ccdproc.trim_image(ccd[:shape[0]-17, :])
    ccd = ccdproc.trim_image(ccd[:, 12:])
    return ccd

In [12]:
def trim_fits_big(arr):
    shape = arr.shape
    ccd = CCDData(arr, unit=u.adu)
    ccd = ccdproc.trim_image(ccd[:shape[0]-34, :])
    ccd = ccdproc.trim_image(ccd[:, 24:])
    return ccd

In [3]:
def rm_overscan(arr):
    shape = arr.shape
    ccd = CCDData(arr, unit=u.adu)
    return ccdproc.subtract_overscan(overscan=im[:shape[0]-34, :])

In [2]:
def bin_image(arr, block_size):
    return ccdproc.block_average(arr, block_size=block_size)

In [6]:
def create_bias(file_path: str, out_name: str, overwrite=True) -> None:
    header_red = False
    data = []
    files = glob.glob(file_path)
    for file in files:
        if header_red == False:
            header = fits.getheader(file)
            header_red == True
        ccd = CCDData(fits.getdata(file), unit=u.adu)
        data.append(ccd)
    combiner = Combiner(data)
    combined_median = combiner.median_combine()
    #combined_median = ccdproc.subtract_overscan(combined_median, overscan=combined_median[12:, :], overscan_axis=None)
    #save_fits(combined_median, out_name, overwrite)
    fits.writeto(out_name, combined_median, header, overwrite=overwrite)

In [8]:
def create_dark(file_path, out_name, overwrite=True, bias_path=None):
    header_red = False
    if bias_path is not None:
        bias = CCDData(fits.getdata(bias_path), unit=u.adu)
        
    data = []
    files = glob.glob(file_path)
    for file in files:
        if header_red == False:
            header = fits.getheader(file)
        ccd = CCDData(fits.getdata(file), unit=u.adu)
        data.append(ccd)
    
    combiner = Combiner(data)
    combined_median = combiner.median_combine()
    #combined_median = ccdproc.subtract_overscan(combined_median, overscan=combined_median[12:, :], overscan_axis=None)
    #save_fits(combined_median, out_name, overwrite)
    
    if bias_path is not None:
        result = combined_median.subtract(bias)
        fits.writeto(out_name, result, header, overwrite=overwrite)
        #save_fits(result, out_name, overwrite)
    else:
        fits.writeto(out_name, combined_median, header, overwrite=overwrite)
        #save_fits(combined_median, out_name, overwrite)

In [20]:
def create_flat(file_path, out_name, overwrite=True, bias_path=None, dark_path=None):
    header_red = False
    bd_remove = False
    if bias_path is not None and dark_path is not None:
        bd_remove = True
        bias_data = fits.getdata(bias_path) 
        dark_data = fits.getdata(dark_path)
        bias_max = np.max(bias_data)
        dark_max = np.max(dark_data)
        dark = CCDData(bias_data, unit=u.adu)
        bias = CCDData(dark_data, unit=u.adu)
    elif bias_path is not None or dark_path is not None:
        print("Error: provide bias and dark images or none")
        exit(1)
        
    data = []
    files = glob.glob(file_path)
    for file in files:
        fits_file = fits.getdata(file)
        if header_red == False:
            header = fits.getheader(file, 0)
            header_red = True
        flat_av = np.mean(fits_file)
        ccd = CCDData(fits_file, unit=u.adu)
        if bd_remove == True:
            ccd = ccd.subtract(bias)
            ccd = ccd.subtract(dark)
        ccd = ccd.divide(flat_av)
        data.append(ccd)
    
    combiner = Combiner(data)
    combined_median = combiner.median_combine()
    #combined_median = ccdproc.subtract_overscan(combined_median, overscan=combined_median[12:, :], overscan_axis=None)
    #save_fits(combined_median, out_name, overwrite)
    #hdulist = combined_median.to_hdu()
    #combined_median.write(out_name, overwrite=True)
    fits.writeto(out_name, combined_median, header, overwrite=overwrite)

In [73]:
def remove_cold_pixels(arr):
    """
    Function removes pixels with negative values, assuming trimmed image.
    """
    #m = np.array([[0, 0, 0],
    #              [0, 1, 0],
    #              [0, 0, 0]])
    shape = arr.shape
    neg_vals = np.where(arr < 0)
    for i, j in zip(neg_vals[0], neg_vals[1]):
        med = arr[i-1:i+2, j-1:j+2]
        #mask_med = ma.array(med, mask=m)
        #median = np.median(mask_med)
        median = np.median(med)
        arr[i, j] = median
    return arr

In [5]:
def remove_hot_pix(arr, hot_pix_file):
    x, y = [], []
    with open(hot_pix_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            text = line.split()
            x.append(int(text[0]))
            y.append(int(text[1]))

    for i, j in zip(x, y):
        med = np.median(arr[i-1:i+2, j-1:j+2])
        arr[i, j] = med
    return arr

In [32]:
def create_reduced_image(image, bias, dark, flat, out_name):
    """
    Function does image reduction and trimming.
    image, bias, dark, flat - paths to images
    """
    header = fits.getheader(image, 0)
    im = CCDData.read(image, unit=u.adu)
    bs = CCDData.read(bias, unit=u.adu)
    dk = CCDData.read(dark, unit=u.adu)
    ft = CCDData.read(flat, unit=u.adu)
    
    im = trim_fits(im)
    bs = trim_fits(bs)
    dk = trim_fits(dk)
    ft = trim_fits(ft)
    #dk.header['exposure'] = 180.0
    bias_subtracted = ccdproc.subtract_bias(im, bs)
    dark_subtracted = ccdproc.subtract_dark(bias_subtracted, dk, exposure_time='EXPOSURE', exposure_unit=u.second, scale=False)
    flat_corr = ccdproc.flat_correct(dark_subtracted, ft, min_value=0.001)
    fits.writeto(out_name, flat_corr, header, overwrite=True)
    reduced = fits.getdata(out_name)
    reduced = np.clip(reduced, 0, 65535, dtype=np.uint16)
    fits.writeto(out_name, flat_corr, header, overwrite=True)
    #save_fits(flat_corr, out_name, True)
    #return flat_corr

In [48]:
def create_reduced_image_big(image, bias, dark, flat, out_name):
    """
    Function does image reduction and trimming.
    image, bias, dark, flat - paths to images
    """
    header = fits.getheader(image, 0)
    im = CCDData.read(image, unit=u.adu)
    bs = CCDData.read(bias, unit=u.adu)
    dk = CCDData.read(dark, unit=u.adu)
    ft = CCDData.read(flat, unit=u.adu)
    
    im = trim_fits_big(im)
    bs = trim_fits_big(bs)
    dk = trim_fits_big(dk)
    ft = trim_fits_big(ft)
    #dk.header['exposure'] = 180.0
    bias_subtracted = ccdproc.subtract_bias(im, bs)
    dark_subtracted = ccdproc.subtract_dark(bias_subtracted, dk, exposure_time='EXPOSURE', exposure_unit=u.second, scale=False)
    flat_corr = ccdproc.flat_correct(dark_subtracted, ft, min_value=0.001)
    fits.writeto(out_name, flat_corr, header, overwrite=True)
    reduced = fits.getdata(out_name)
    reduced = np.clip(reduced, 0, 65535)
    reduced = reduced.astype(np.uint16)
    #reduced = remove_cold_pixels(reduced)
    fits.writeto(out_name, reduced, header, overwrite=True)
    #save_fits(flat_corr, out_name, True)
    #return flat_corr